In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ELT_PySpark") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "your username") \
    .config("spark.hadoop.fs.s3a.secret.key", "your password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/23 13:26:22 WARN Utils: Your hostname, lenovo-PC resolves to a loopback address: 127.0.1.1; using 172.31.112.46 instead (on interface eth0)
25/05/23 13:26:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntucoy/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntucoy/.ivy2/cache
The jars for the packages stored in: /home/ubuntucoy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-efe7c878-a5fe-428d-ba1c-a6033c9b0113;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 2802ms :: artifacts dl 107ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.11.1026] in [default]
	----------------------------------------------------

In [2]:
df = spark.read.csv("s3a://raw/crm/sales_details.csv", header=True, inferSchema=True)
df.show()

25/05/23 13:28:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|sls_ord_num|sls_prd_key|sls_cust_id|sls_order_dt|sls_ship_dt|sls_due_dt|sls_sales|sls_quantity|sls_price|
+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|    SO43697| BK-R93R-62|      21768|    20101229|   20110105|  20110110|     3578|           1|     3578|
|    SO43698| BK-M82S-44|      28389|    20101229|   20110105|  20110110|     3400|           1|     3400|
|    SO43699| BK-M82S-44|      25863|    20101229|   20110105|  20110110|     3400|           1|     3400|
|    SO43700| BK-R50B-62|      14501|    20101229|   20110105|  20110110|      699|           1|      699|
|    SO43701| BK-M82S-44|      11003|    20101229|   20110105|  20110110|     3400|           1|     3400|
|    SO43702| BK-R93R-44|      27645|    20101230|   20110106|  20110111|     3578|           1|     3578|
|    SO43703| BK-R93R-62|      16624|

In [3]:
from pyspark.sql.functions import (col, length, when, to_date, abs, expr)

# Transformation 
df_transformed = df.select(
    "sls_ord_num",
    "sls_prd_key",
    "sls_cust_id",
    when(
        (col("sls_order_dt") == 0) | (length(col("sls_order_dt").cast("string")) != 8),
        None
    ).otherwise(
        to_date(col("sls_order_dt").cast("string"), "yyyyMMdd")
    ).alias("sls_order_dt"),                                   # Date transformation: sls_order_dt
    when(
        (col("sls_ship_dt") == 0) | (length(col("sls_ship_dt").cast("string")) != 8),
        None
    ).otherwise(
        to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd")
    ).alias("sls_ship_dt"),                                    # Date transformation: sls_ship_dt
    when(
        (col("sls_due_dt") == 0) | (length(col("sls_due_dt").cast("string")) != 8),
        None
    ).otherwise(
        to_date(col("sls_due_dt").cast("string"), "yyyyMMdd")
    ).alias("sls_due_dt"),                                     # Date transformation: sls_due_dt
    when(
        col("sls_sales").isNull() |
        (col("sls_sales") <= 0) |
        (col("sls_sales") != col("sls_quantity") * abs(col("sls_price"))),
        col("sls_quantity") * abs(col("sls_price"))
    ).otherwise(
        col("sls_sales")
    ).alias("sls_sales"),                                      # sls_sales: recalculate if NULL, <=0, or does not match formula
    "sls_quantity",
    when(
        col("sls_price").isNull() | (col("sls_price") <= 0),
        expr("sls_sales / NULLIF(sls_quantity, 0)")
    ).otherwise(
        col("sls_price")
    ).alias("sls_price")                                       # sls_price: derive if NULL or <=0
)
df_transformed.show()

[Stage 3:>                                                          (0 + 1) / 1]

+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|sls_ord_num|sls_prd_key|sls_cust_id|sls_order_dt|sls_ship_dt|sls_due_dt|sls_sales|sls_quantity|sls_price|
+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|    SO43697| BK-R93R-62|      21768|  2010-12-29| 2011-01-05|2011-01-10|     3578|           1|   3578.0|
|    SO43698| BK-M82S-44|      28389|  2010-12-29| 2011-01-05|2011-01-10|     3400|           1|   3400.0|
|    SO43699| BK-M82S-44|      25863|  2010-12-29| 2011-01-05|2011-01-10|     3400|           1|   3400.0|
|    SO43700| BK-R50B-62|      14501|  2010-12-29| 2011-01-05|2011-01-10|      699|           1|    699.0|
|    SO43701| BK-M82S-44|      11003|  2010-12-29| 2011-01-05|2011-01-10|     3400|           1|   3400.0|
|    SO43702| BK-R93R-44|      27645|  2010-12-30| 2011-01-06|2011-01-11|     3578|           1|   3578.0|
|    SO43703| BK-R93R-62|      16624|

In [4]:
df_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("s3a://clean/crm/sales_details_clean.csv")

25/05/23 13:29:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/23 13:30:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
                                                                                

In [5]:
df_cleaned = spark.read.option("header", True).csv("s3a://clean/crm/sales_details_clean.csv")
df_cleaned.show()

+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|sls_ord_num|sls_prd_key|sls_cust_id|sls_order_dt|sls_ship_dt|sls_due_dt|sls_sales|sls_quantity|sls_price|
+-----------+-----------+-----------+------------+-----------+----------+---------+------------+---------+
|    SO43697| BK-R93R-62|      21768|  2010-12-29| 2011-01-05|2011-01-10|     3578|           1|   3578.0|
|    SO43698| BK-M82S-44|      28389|  2010-12-29| 2011-01-05|2011-01-10|     3400|           1|   3400.0|
|    SO43699| BK-M82S-44|      25863|  2010-12-29| 2011-01-05|2011-01-10|     3400|           1|   3400.0|
|    SO43700| BK-R50B-62|      14501|  2010-12-29| 2011-01-05|2011-01-10|      699|           1|    699.0|
|    SO43701| BK-M82S-44|      11003|  2010-12-29| 2011-01-05|2011-01-10|     3400|           1|   3400.0|
|    SO43702| BK-R93R-44|      27645|  2010-12-30| 2011-01-06|2011-01-11|     3578|           1|   3578.0|
|    SO43703| BK-R93R-62|      16624|

In [6]:
import duckdb

# Convert from Spark to Pandas
df_cleaned_sales_details = df_cleaned.toPandas()

# Connect to DuckDB
con = duckdb.connect("/mnt/d/data_engineering/duckdb/db/dev.duckdb")

# Save as DuckDB table (view)
con.register("crm_sales_details_clean_view", df_cleaned_sales_details)

# Save as DuckDB table (not view)
con.execute("CREATE OR REPLACE TABLE crm_sales_details_clean AS SELECT * FROM crm_sales_details_clean_view")

In [7]:
con.execute("SELECT * FROM crm_sales_details_clean LIMIT 10").df().style.hide(axis="index")

sls_ord_num,sls_prd_key,sls_cust_id,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price
SO43697,BK-R93R-62,21768,2010-12-29,2011-01-05,2011-01-10,3578,1,3578.0
SO43698,BK-M82S-44,28389,2010-12-29,2011-01-05,2011-01-10,3400,1,3400.0
SO43699,BK-M82S-44,25863,2010-12-29,2011-01-05,2011-01-10,3400,1,3400.0
SO43700,BK-R50B-62,14501,2010-12-29,2011-01-05,2011-01-10,699,1,699.0
SO43701,BK-M82S-44,11003,2010-12-29,2011-01-05,2011-01-10,3400,1,3400.0
SO43702,BK-R93R-44,27645,2010-12-30,2011-01-06,2011-01-11,3578,1,3578.0
SO43703,BK-R93R-62,16624,2010-12-30,2011-01-06,2011-01-11,3578,1,3578.0
SO43704,BK-M82B-48,11005,2010-12-30,2011-01-06,2011-01-11,3375,1,3375.0
SO43705,BK-M82S-38,11011,2010-12-30,2011-01-06,2011-01-11,3400,1,3400.0
SO43706,BK-R93R-48,27621,2010-12-31,2011-01-07,2011-01-12,3578,1,3578.0


In [8]:
con.close()

25/05/23 14:02:55 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 276045 ms exceeds timeout 120000 ms
25/05/23 14:02:58 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/23 14:03:02 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$